In [1]:
import sys,os
sys.path.append("/home/paco/Documents/site_similarity")

In [2]:
from utils.notebook_utils import load_corpus

In [3]:
old_data = load_corpus('modified_corpus.csv')

In [4]:
import pandas as pd

pd.DataFrame(old_data).head()

,source_url,source_url_processed,URL,fact,bias
0,http://www.villagevoice.com/,villagevoice.com,http://mediabiasfactcheck.com/the-village-voice/,HIGH,left
1,https://insideclimatenews.org/,insideclimatenews.org,https://mediabiasfactcheck.com/insideclimate-n...,HIGH,left-center
2,http://www.fury.news/,fury.news,http://mediabiasfactcheck.com/fury-news/,LOW,extreme-right
3,http://now8news.com/,now8news.com,http://mediabiasfactcheck.com/now8news/,LOW,center
4,http://constitution.com/,constitution.com,http://mediabiasfactcheck.com/the-constitution/,LOW,extreme-right


In [5]:
new_data = load_corpus('new_corpus_2020.csv')

In [6]:
pd.DataFrame(new_data).head()

,source_url,source_url_normalized,ref,fact,bias
0,https://crooked.com,crooked.com,https://mediabiasfactcheck.com/crooked-media/,high,left
1,http://deepleftfield.info,deepleftfield.info,https://mediabiasfactcheck.com/deep-left-field/,mixed,left
2,https://antifascistnews.net,antifascistnews.net,https://mediabiasfactcheck.com/anti-fascist-news/,high,left
3,http://www.cnn.com,cnn.com,http://mediabiasfactcheck.com/cnn/,mixed,left
4,http://www.allthatsfab.com,allthatsfab.com,http://mediabiasfactcheck.com/all-thats-fab/,mixed,left


In [7]:
print(f"Old data: {len(old_data)} New data: {len(new_data)}")

Old data: 1066 New data: 859


In [8]:
old_urls = set(row['source_url_processed'] for row in old_data)

list(old_urls)[:10]

['tmn.today',
 'rstreet.org',
 'thedailyvox.co.za',
 'thedailyliberator.com',
 'personalliberty.com',
 'thewest.com.au',
 'toinformistoinfluence.com',
 'freakoutnation.com',
 'msnbc.com',
 'retractionwatch.com']

In [9]:
new_urls = set(row['source_url_normalized'] for row in new_data)

list(new_urls)[:10]

['tmn.today',
 'thedailyvox.co.za',
 'freakoutnation.com',
 'msnbc.com',
 'personalliberty.com',
 'thatsnonsense.com',
 'conservativeopinion.com',
 'defenddemocracy.org',
 'disobedientmedia.com',
 'dallasvoice.com']

In [10]:
len(old_urls.intersection(new_urls))

513

In [11]:
print(f"Urls that are in both sets: {len(old_urls & new_urls)}")

Urls that are in both sets: 513


In [12]:
print(f"Urls that are not in old sets: {len(new_urls - old_urls)}")

Urls that are not in old sets: 346


# Start Scrapping missing elements from new corpus

In [13]:
from dataprep.alexa_scrapper import ScrapeAlexa
from tqdm import tqdm

results = {}
alexa_scrapper = ScrapeAlexa()
for row in tqdm(new_data):
    results[row['source_url_normalized']] = {row['source_url_normalized']: alexa_scrapper.scrape_alexa_site_info(row['source_url_normalized'])}

100%|██████████| 859/859 [04:39<00:00,  3.07it/s]


In [14]:
[res for result in results.values() for res in result.values() if res['score'] is None]

[]

In [15]:
results['newpol.org']

{'newpol.org': {'site': 'newpol.org',
  'score': [{'url': 'leftvoice.org',
    'overlap_score': 2.1,
    'alexa_rank': 131685.0},
   {'url': 'internationalviewpoint.org',
    'overlap_score': 2.1,
    'alexa_rank': 516782.0},
   {'url': 'isreview.org', 'overlap_score': 1.9, 'alexa_rank': 427457.0},
   {'url': 'solidarity-us.org', 'overlap_score': 1.8, 'alexa_rank': 640525.0},
   {'url': 'isj.org.uk', 'overlap_score': 1.8, 'alexa_rank': 805492.0}],
  'audience_geography': [{'country': 'Argentina', 'percent': 69.4},
   {'country': 'United States', 'percent': 18.1}]}}

In [16]:
# Distribution of the results length

from collections import Counter

Counter([len(res['score']) for result in results.values() for res in result.values()])

Counter({5: 794, 2: 8, 1: 13, 0: 29, 3: 4, 4: 11})

In [18]:
import json

with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 0 results.json", 'w') as f:
    json.dump(results, f, indent=4)

# Level one scrapping

In [19]:
import json
with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 0 results.json") as f:
    lvl_zero_data = json.load(f)

In [21]:
from dataprep.alexa_scrapper import scrapping


lvl_one_file = "/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 1 results.json"

results = scrapping(lvl_zero_data, output_file=lvl_one_file)

100%|██████████| 859/859 [48:00<00:00,  3.35s/it]


In [22]:
[res for result in results.values() for res in result.values() if res['score'] is None]

[]

In [23]:
with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 1 results.json", 'w') as f:
    json.dump(results, f, indent=4)

# Level two scrapping

In [35]:
with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 1 results.json") as f:
    lvl_one_data = json.load(f)

In [36]:
len(lvl_one_data)

859

In [37]:
lvl_two_file = "/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 2 results.json"

results = scrapping(lvl_one_data, output_file=lvl_two_file)

100%|██████████| 859/859 [00:30<00:00, 28.27it/s]


In [38]:
[res for result in results.values() for res in result.values() if res['score'] is None]

[]

In [39]:
len(results)

2933

In [40]:
with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 2 results.json", 'w') as f:
    json.dump(results, f, indent=4)

# Level 3 scrapping

In [46]:
with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 2 results.json") as f:
    lvl_two_data = json.load(f)

In [47]:
len(lvl_two_data)

2933

In [59]:
lvl_three_file = "/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 3 results.json"

results = scrapping(lvl_two_data,output_file=lvl_three_file)

100%|██████████| 2933/2933 [02:34<00:00, 19.02it/s]


In [60]:
[res for result in results.values() for res in result.values() if res['score'] is None]

[]

In [61]:
len(results)

9101

In [62]:
with open(lvl_three_file, 'w') as f:
    json.dump(results, f, indent=4)

# Level 4 scrappimg

In [63]:
with open("/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 3 results.json") as f:
    lvl_three_data = json.load(f)

In [64]:
len(lvl_three_data)

9101

In [66]:
[res for result in lvl_three_data.values() for res in result.values() if res['score'] is None]

[]

In [67]:
lvl_four_file = "/home/paco/Documents/site_similarity/dataset/scrapping_results/new corpus processed level 4 results.json"

results = scrapping(lvl_three_data,output_file=lvl_four_file)

 36%|███▌      | 3268/9101 [8:21:48<14:55:40,  9.21s/it]


ConnectionError: HTTPSConnectionPool(host='www.alexa.com', port=443): Max retries exceeded with url: /siteinfo/mrplasko-psych-soc.wikispaces.com (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcc415035c0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
[res for result in results.values() for res in result.values() if res['score'] is None]